In [1]:
import pandas as pd
import numpy as np

import miditoolkit

import re
import glob
import os

from music21 import converter, midi, key, instrument, meter, interval, note, pitch, stream, tempo, chord
import os
import pandas as pd
import glob

from tqdm import tqdm

from utils.utils import open_midi, get_file_name, extract_tempo, extract_key, to_snake_case, chord_type_checker

from difflib import get_close_matches

In [2]:
p_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/program_change.csv')
com_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/commu_meta.csv', index_col=0)
df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/allgenre_cp.csv')

chord_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/chord_list.csv')

In [3]:
scale = ["maj", "maj7","7","dim","dim7","+","m","m7","sus4","7sus4","m7b5","sus2","add2","madd2","mM7", '5', '6' , 'm5' , 'm6']
chord_df['CHORD_TYPE'] = chord_df['CHORD_TYPE'].apply(lambda x: '+' if x == 'aug' else x)
chord_df['CHORD_TYPE'] = chord_df['CHORD_TYPE'].apply(lambda x: 'mM7' if x == 'm(maj7)' else x)
chord_df['CHORD_TYPE'] = chord_df['CHORD_TYPE'].apply(lambda x: 'add2' if x == 'add9' else x)
chord_df = chord_df[chord_df['CHORD_TYPE'].isin(scale)]
chord_df.reset_index(drop=True, inplace=True)

chord_df['Notes_list'] = chord_df['NOTES'].apply(lambda x: [y for y in x.split(',')])
chord_df['Notes_list_string'] = chord_df['Notes_list'].apply(lambda x: str(x))

In [4]:
commu_folder = './commu_midi'
com_m = glob.glob(f'{commu_folder}/*.mid')
com_m.sort()

In [5]:
our_m = list()
for idx, v in enumerate(df['song_title'].values):

    genre = df['genre'].values[idx]
    data_folder = './data/' + genre + '/cropped'

    our_m.append(os.path.join(data_folder, v + '/' + df['file_name'].values[idx]))

In [6]:
# score = converter.parse(our_m[60396])
# chords = df['chord_progressions'][60396]

In [7]:
df[1964:1970]

,song_title,file_name,program_change_value,inst,start_position,end_position,num_measures,bpm,key,instrument_name,repeated,genre,min_pitch,max_pitch,mean_pitch,min_velocity,max_velocity,chord_progressions,pitch_range,time_signature
1964,AllOfMe,02_vibes_02.mid,11,vibraphone,17,24,8,135,c_major,02_vibes,NaN,movie,63,76,69,81,81,"['e4', 'g4', 'e4', 'f4', 'g#4', 'b4', 'e5', 'b...",mid,4/4
1965,AllOfMe,02_vibes_03.mid,11,vibraphone,25,32,8,135,c_major,02_vibes,NaN,movie,63,74,68,81,81,"['e4', 'c5', 'c#5', 'd5', 'c5', 'c#5', 'd5', '...",mid,4/4
1966,AllOfMe,02_vibes_04.mid,11,vibraphone,33,40,8,135,c_major,02_vibes,NaN,movie,62,76,70,81,81,"['d5', 'c5', 'b4', 'd4', 'g4', 'a4', 'b4', 'b4...",mid,4/4
1967,AllOfMe,02_vibes_08.mid,11,vibraphone,65,72,8,135,c_major,02_vibes,NaN,movie,63,84,72,81,81,"['c6', 'a5', 'g#5', 'g5', 'e5', 'f5', 'e-5', '...",mid,4/4
1968,AllOfMe,02_vibes_09.mid,11,vibraphone,73,80,8,135,c_major,02_vibes,NaN,movie,63,74,68,81,81,"['e4', 'c5', 'd5', 'c5', 'b4', 'b4', 'g#4', 'e...",mid,4/4
1969,AllOfMe,02_vibes_10.mid,11,vibraphone,81,88,8,135,c_major,02_vibes,NaN,movie,64,76,71,81,81,"['d5', 'b4', 'c5', 'c#5', 'd5', 'b4', 'c5', 'd...",mid,4/4


In [8]:
cl = chord_df['NOTES'].to_list()
for idx, v in enumerate(cl):
    cl[idx] = v.replace(',', '')

for idx, v in enumerate(cl):
    cl[idx] = v.replace('b', '-')

In [9]:
our_m[1964]

'./data/movie/cropped/AllOfMe/02_vibes_02.mid'

In [10]:
score = converter.parse(our_m[1964])
score.show('text')

{0.0} <music21.metadata.Metadata object at 0x14fe84ac0>
{0.0} <music21.stream.Part 0x14fe84a30>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.instrument.Vibraphone 'Vibes: Vibes'>
        {0.0} <music21.instrument.Vibraphone 'Vibraphone'>
        {0.0} <music21.instrument.Vibraphone 'Vibraphone'>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.tempo.MetronomeMark animato Quarter=120.0>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.note.Rest 2/3ql>
        {0.6667} <music21.note.Note E>
        {1.0} <music21.note.Note G>
        {1.6667} <music21.note.Note E>
        {2.0} <music21.note.Note F>
        {2.3333} <music21.note.Note G#>
        {2.6667} <music21.note.Note B>
        {3.0} <music21.note.Note E>
        {3.6667} <music21.note.Note B>
    {4.0} <music21.stream.Measure 2 offset=4.0>
        {0.0} <music21.note.Note D>
        {0.6667} <music21.note.Note C>
    {8.0} <music21.stream.Measure 3 offset=8.0>
        

In [11]:
chord_progression = list()
cp = list()

for i in range(8):
    
    note_list = list()

    for obj in score.measures(i+1, i+1).recurse():
        if isinstance(obj, note.Note):
            if obj.duration.quarterLength > 0.3:
                note_list.append(obj.name)  
        elif isinstance(obj, chord.Chord):
            for p in obj.pitches:   
                note_list.append(p.name)
                
    print('Measuring: ', i+1)

    current_chord = list()

    for value in note_list:
        if value not in current_chord:
            current_chord.append(value)

    if len(current_chord) == 0:
        cp.append('Bar_None')

    if len(current_chord) == 1:
        cp.append(str(current_chord[0]))

    if len(current_chord) > 1:

        notes = str()
        
        for i in current_chord:
            notes += i
        
        print(notes)
        close_matches = get_close_matches(notes, cl , 1, 0.5)
        print(close_matches)

        tnl = list()
        for i in range(len(str(close_matches[0]))):
            if close_matches[0][i] == '#':
                tnl.append(close_matches[0][i-1] + '#')
                tnl.remove(close_matches[0][i-1])
            elif close_matches[0][i] == '-':
                tnl.append(close_matches[0][i-1] + 'b')
                tnl.remove(close_matches[0][i-1])
            else:
                tnl.append(close_matches[0][i])

        basic = chord_df[chord_df['Notes_list_string'] == str (tnl)]['CHORD_ROOT'].values[0]
        chord_type = chord_df[chord_df['Notes_list_string'] == str(tnl)]['CHORD_TYPE'].values[0]
        if chord_type == '5':
            chord_type = ''
        if chord_type == 'm5':
            chord_type = 'm'
        cp.append(str(basic + chord_type))

Measuring:  1
EGFG#B
['EG#B']
Measuring:  2
DC
['DGAC']
Measuring:  3
Measuring:  4
BB-AG
['B-DFG']
Measuring:  5
B-ADB
['GB-D']
Measuring:  6
AG
['FACG']
Measuring:  7
Measuring:  8
BCF#GE-
['CFGB-']


In [12]:
note_list = list()

for obj in score.measures(4, 4).recurse():
    if isinstance(obj, note.Note):
        if obj.duration.quarterLength > 0.3:
            note_list.append(obj.name)  
    elif isinstance(obj, chord.Chord):
        for p in obj.pitches:   
            note_list.append(p.name) 

note_list

['B', 'B-', 'A', 'G']

In [13]:
current_chord = list()

for value in note_list:
    if value not in current_chord:
        current_chord.append(value)
        
current_chord

['B', 'B-', 'A', 'G']

In [14]:
notes = str()
for i in current_chord:
    notes += i

# for i in notes:
#     if i == '-':
#         notes = notes.replace('-', 'o')
notes

'BB-AG'

In [15]:
close_matches = get_close_matches(notes, cl , 1, 0.5)

close_matches

['B-DFG']

In [18]:

tnl = list()
for i in range(len(str(close_matches[0]))):
    if close_matches[0][i] == '#':
        tnl.append(close_matches[0][i-1] + '#')
        tnl.remove(close_matches[0][i-1])
    elif close_matches[0][i] == '-':
        tnl.append(close_matches[0][i-1] + 'b')
        tnl.remove(close_matches[0][i-1])
    else:
        tnl.append(close_matches[0][i])

print(tnl)

chord_df[chord_df['Notes_list_string'] == str(tnl)]

['Bb', 'D', 'F', 'G']


,CHORD_ROOT,CHORD_TYPE,CHORD_STRUCTURE,NOTES,Notes_list,Notes_list_string
147,Bb,6,1;3;5;6,"Bb,D,F,G","[Bb, D, F, G]","['Bb', 'D', 'F', 'G']"


In [130]:
def extract_chord_progression(midi_path, chord_df, cl):

    score = converter.parse(midi_path)

    chord_progression = list()
    cp = list()

    for i in range(8):
        
        note_list = list()

        for obj in score.measures(i+1, i+1).recurse():
            if isinstance(obj, note.Note):
                if obj.duration.quarterLength > 0.3:
                    note_list.append(obj.name)  
            elif isinstance(obj, chord.Chord):
                for p in obj.pitches:   
                    note_list.append(p.name) 

        current_chord = list()

        for value in note_list:
            if value not in current_chord:
                current_chord.append(value)

        if len(current_chord) == 0:
            cp.append('Bar_None')

        if len(current_chord) == 1:
            cp.append(str(current_chord[0]))

        if len(current_chord) > 1:

            notes = str()
            
            for i in current_chord:
                notes += i
            
            if len(notes) > 5:
                notes = notes[:5]
            
            close_matches = get_close_matches(notes, cl , 1, 0.6)
            
            if not close_matches:
                close_matches = get_close_matches(notes, cl , 1, 0.5)
                print(midi_path, close_matches)

            tnl = list()
            for i in range(len(str(close_matches[0]))):
                if close_matches[0][i] == '#':
                    tnl.append(close_matches[0][i-1] + '#')
                    tnl.remove(close_matches[0][i-1])
                elif close_matches[0][i] == '-':
                    tnl.append(close_matches[0][i-1] + 'b')
                    tnl.remove(close_matches[0][i-1])
                else:
                    tnl.append(close_matches[0][i])

            basic = chord_df[chord_df['Notes_list_string'] == str (tnl)]['CHORD_ROOT'].values[0]
            chord_type = chord_df[chord_df['Notes_list_string'] == str(tnl)]['CHORD_TYPE'].values[0]
            if chord_type == '5':
                chord_type = ''
            if chord_type == 'm5':
                chord_type = 'm'
            cp.append(str(basic + chord_type))

    for c in cp:
        for i in range(8):
            chord_progression.append(c)
    
    return chord_progression

In [131]:
df['chord_progressions'] = None

cl = chord_df['NOTES'].to_list()
for idx, v in enumerate(cl):
    cl[idx] = v.replace(',', '')

for idx, v in enumerate(cl):
    cl[idx] = v.replace('b', '-')

for idx, path in tqdm(enumerate(our_m)):
    
    chord_progression = extract_chord_progression(path, chord_df, cl)
    df.at[idx, 'chord_progressions'] = [chord_progression]
    

31it [00:00, 83.19it/s] 

./data/movie/cropped/10/01_stringinstrument_09.mid ['GAD']
./data/movie/cropped/10/01_stringinstrument_09.mid ['GCD']
./data/movie/cropped/10/01_stringinstrument_10.mid ['FGC']
./data/movie/cropped/10/01_stringinstrument_10.mid ['FGC']
./data/movie/cropped/10/01_stringinstrument_10.mid ['BDF']
./data/movie/cropped/10/01_stringinstrument_13.mid ['DFA']
./data/movie/cropped/10/01_stringinstrument_13.mid ['EGBD']
./data/movie/cropped/10/01_stringinstrument_14.mid ['EGBD']
./data/movie/cropped/10/01_stringinstrument_14.mid ['EAB']


50it [00:00, 83.30it/s]

./data/movie/cropped/101Dalmations/00_brass_00.mid ['FAC']
./data/movie/cropped/101Dalmations/00_brass_02.mid ['EGB']
./data/movie/cropped/101Dalmations/00_brass_02.mid ['EGB']
./data/movie/cropped/101Dalmations/00_brass_06.mid ['EGB']
./data/movie/cropped/101Dalmations/00_brass_06.mid ['EGB']
./data/movie/cropped/101Dalmations/01_electric_piano_00.mid ['FAC']
./data/movie/cropped/101Dalmations/01_electric_piano_02.mid ['GCD']
./data/movie/cropped/101Dalmations/01_electric_piano_02.mid ['EG#BF#']


59it [00:00, 63.69it/s]

./data/movie/cropped/101Dalmations/01_electric_piano_06.mid ['GCD']


83it [00:01, 60.25it/s]


./data/movie/cropped/12Monkeys/04_stringinstrument_02.mid []


IndexError: list index out of range